In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semeval2023-task10/train_aug.csv
/kaggle/input/semeval2023-task10/dev_task_a_entries.csv
/kaggle/input/semeval2023-task10/test_task_a_entries.csv
/kaggle/input/semeval2023-task10/edos_labelled_aggregated.csv
/kaggle/input/semeval2023-task10/train_all_tasks.csv


In [49]:
%%capture c
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [50]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

In [71]:
def process_data(row, max_len):
    text = ' '.join(str(row['text']).split())
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=max_len)
    label = 1 if row['label_sexist'] == 'sexist' else 0
    encodings['label'] = label
    encodings['text'] = text
    return encodings


def get_prediction(text, max_len):
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_len)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = model(**encoding)
    logits = outputs.logits
    probs =  F.softmax(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    return {'label_sexist': np.argmax(probs, axis=-1),
            'probability': probs}

def get_probabilites_dev(model_name, val):
    taska_preds = pd.read_csv('/kaggle/input/semeval2023-task10/dev_task_a_entries.csv')[["rewire_id", "text"]]
    taska_preds.columns = ["rewire_id", "label_pred"]
    tqdm.pandas()
    taska_preds["label_pred"] = taska_preds["label_pred"].progress_apply(lambda text: get_prediction(text, val["max_len"]))
    taska_preds["label"] = ["sexist" if i["label_sexist"] == 1 else "not sexist" for i in taska_preds.label_pred.values]
    taska_preds["probability"] = [i["probability"] for i in taska_preds.label_pred.values]
    taska_preds["not sexist"] = [i[0] for i in taska_preds["probability"] ]
    taska_preds["sexist"] = [i[1] for i in taska_preds["probability"] ]
    taska_preds["label_pred"] = taska_preds["label"] 
    del  taska_preds["probability"] 
    del  taska_preds["label"] 
    taska_preds.to_csv(f"dev_{model_name}_maxlen_{val['max_len']}_batch{val['batch_size']}_lr_{val['learning_rate']}.csv", index=False)

def get_probabilites_test(model_name, val):
    taska_preds = pd.read_csv('/kaggle/input/semeval2023-task10/test_task_a_entries.csv')
    taska_preds.columns = ["rewire_id", "label_pred"]
    tqdm.pandas()
    taska_preds["label_pred"] = taska_preds["label_pred"].progress_apply(lambda text: get_prediction(text, val["max_len"]))
    taska_preds["label"] = ["sexist" if i["label_sexist"] == 1 else "not sexist" for i in taska_preds.label_pred.values]
    taska_preds["probability"] = [i["probability"] for i in taska_preds.label_pred.values]
    taska_preds["not sexist"] = [i[0] for i in taska_preds["probability"] ]
    taska_preds["sexist"] = [i[1] for i in taska_preds["probability"] ]
    taska_preds["label_pred"] = taska_preds["label"] 
    del  taska_preds["probability"] 
    del  taska_preds["label"] 
    taska_preds.to_csv(f"test_{model_name}_maxlen_{val['max_len']}_batch{val['batch_size']}_lr_{val['learning_rate']}.csv", index=False)


In [74]:
# df = pd.read_csv('/kaggle/input/semeval2023-task10/train_all_tasks.csv')[["text", "label_sexist"]]
# df = df.append(dev).reset_index(drop=True)

models = { 

"debertav3base1" : {"model": "microsoft/deberta-v3-base", "max_len": 50, "batch_size": 32, "learning_rate":5e-5},
"debertav3base2" : {"model": "microsoft/deberta-v3-base", "max_len": 60, "batch_size": 32, "learning_rate":5e-5},
"debertav3base3" : {"model": "microsoft/deberta-v3-base", "max_len": 70, "batch_size": 32, "learning_rate":5e-5},
"debertav3base4" : {"model": "microsoft/deberta-v3-base", "max_len": 80, "batch_size": 32, "learning_rate":5e-5},
"debertav3base5" : {"model": "microsoft/deberta-v3-base", "max_len": 90, "batch_size": 32, "learning_rate":5e-5},
"debertav3base6" : {"model": "microsoft/deberta-v3-base", "max_len": 100, "batch_size": 32, "learning_rate":5e-5},
"debertav3base7" : {"model": "microsoft/deberta-v3-base", "max_len": 110, "batch_size": 32, "learning_rate":5e-5},

"bertracism3_3" : {"model": "MutazYoune/bert_racism3", "max_len": 80, "batch_size": 32, "learning_rate":3e-5},
"bertracism3_4" : {"model": "MutazYoune/bert_racism3", "max_len": 90, "batch_size": 32, "learning_rate":3e-5},
"bertracism3_5" : {"model": "MutazYoune/bert_racism3", "max_len": 100, "batch_size": 32, "learning_rate":3e-5},
"bertracism3_6" : {"model": "MutazYoune/bert_racism3", "max_len": 110, "batch_size": 32, "learning_rate":3e-5},

"bertracism4_2" : {"model": "MutazYoune/bert_racism4", "max_len": 70, "batch_size": 32, "learning_rate":3e-5},
"bertracism4_3" : {"model": "MutazYoune/bert_racism4", "max_len": 80, "batch_size": 32, "learning_rate":3e-5},
"bertracism4_4" : {"model": "MutazYoune/bert_racism4", "max_len": 90, "batch_size": 32, "learning_rate":3e-5},
"bertracism4_5" : {"model": "MutazYoune/bert_racism4", "max_len": 100, "batch_size": 32, "learning_rate":3e-5},
"bertracism4_6" : {"model": "MutazYoune/bert_racism4", "max_len": 110, "batch_size": 32, "learning_rate":3e-5},

"hateracism90000" : {"model": "MutazYoune/bert_hateracism90000", "max_len": 90, "batch_size": 32, "learning_rate":5e-5},
"hateBERT" : {"model": "GroNLP/hateBERT", "max_len": 90, "batch_size": 32, "learning_rate":5e-5},
    
}

In [75]:
from transformers import AutoTokenizer
import pyarrow as pa
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import DebertaV2ForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch


for model_name, val in  models.items():
    df = pd.read_csv('/kaggle/input/semeval2023-task10/train_all_tasks.csv')[["text", "label_sexist"]]

    tokenizer = AutoTokenizer.from_pretrained(val["model"])
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    processed_data = []
    for i in range(len(df)):
        processed_data.append(process_data(df.iloc[i], val["max_len"]))
    train_df = pd.DataFrame(processed_data)
    train_hg = Dataset(pa.Table.from_pandas(train_df))

    model = AutoModelForSequenceClassification.from_pretrained(val["model"], num_labels=2, ignore_mismatched_sizes=True)

    training_args = TrainingArguments(output_dir="./result", evaluation_strategy="no",
                                      num_train_epochs=3, learning_rate = val["learning_rate"], per_device_train_batch_size=val["batch_size"])
    trainer = Trainer(model=model,args=training_args,train_dataset=train_hg,tokenizer=tokenizer)
    trainer.train()

    device = torch.device("cuda")
    model.cuda()
    get_probabilites_dev(model_name, val)
    get_probabilites_test(model_name, val)

loading configuration file https://huggingface.co/microsoft/deberta-v3-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e6f9db57345f0f60c9f837fa97bcb27b1ed31e99feb33d732d7d8c80cb8f8459.de97182a9f32a68819030ba8f3f6ff2ba47276be3864425925523202f54cc79c
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_

Step,Training Loss
500,0.366300
1000,0.213300


Saving model checkpoint to ./result/checkpoint-500
Configuration saved in ./result/checkpoint-500/config.json
Model weights saved in ./result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./result/checkpoint-1000
Configuration saved in ./result/checkpoint-1000/config.json
Model weights saved in ./result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


  0%|          | 0/2000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import k

Step,Training Loss
500,0.367700
1000,0.217700


Saving model checkpoint to ./result/checkpoint-500
Configuration saved in ./result/checkpoint-500/config.json
Model weights saved in ./result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./result/checkpoint-1000
Configuration saved in ./result/checkpoint-1000/config.json
Model weights saved in ./result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


  0%|          | 0/2000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import k

...

storing https://huggingface.co/GroNLP/hateBERT/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/11f5da32001370f4754cfc3a58f35f0f4d957be3460bfa723d54f2a723b310f3.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/11f5da32001370f4754cfc3a58f35f0f4d957be3460bfa723d54f2a723b310f3.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/GroNLP/hateBERT/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/49d465421d47e594ee91d545384e05f9983710fbcf60a70ee1eedda2f9421d3d.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/GroNLP/hateBERT/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/GroNLP/hateBERT/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/GroNLP/hateBERT/resolve/main/special_tokens_map

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/GroNLP/hateBERT/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/2d60dac9cab334b55695ad87a4ab6ec3f4a6997076bdf05c35e02244152f20d6.f37afc087294d5ecdc0a69dd12092647a6e439435e9a0a4a790b086adb5b05f6
creating metadata file for /root/.cache/huggingface/transformers/2d60dac9cab334b55695ad87a4ab6ec3f4a6997076bdf05c35e02244152f20d6.f37afc087294d5ecdc0a69dd12092647a6e439435e9a0a4a790b086adb5b05f6
loading weights file https://huggingface.co/GroNLP/hateBERT/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/2d60dac9cab334b55695ad87a4ab6ec3f4a6997076bdf05c35e02244152f20d6.f37afc087294d5ecdc0a69dd12092647a6e439435e9a0a4a790b086adb5b05f6
Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.p

Step,Training Loss
500,0.324700
1000,0.137200


Saving model checkpoint to ./result/checkpoint-500
Configuration saved in ./result/checkpoint-500/config.json
Model weights saved in ./result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./result/checkpoint-1000
Configuration saved in ./result/checkpoint-1000/config.json
Model weights saved in ./result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


  0%|          | 0/2000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import k